# Word2Vec augmentations line

In [1]:
import pandas as pd
import numpy as np
import re

Подгрузить данные

In [2]:
df_pos = pd.read_csv('../../data/twitts/positive.csv', sep=';', header=None).assign(positive=1)
df_neg = pd.read_csv('../../data/twitts/negative.csv', sep=';', header=None).assign(positive=0)

Объединить

In [3]:
tweets = df_pos[[3,'positive']].append( df_neg[[3,'positive']], ignore_index=True ).rename({3:'tweet'}, axis=1)

In [4]:
tweets.head(15)

,tweet,positive
0,"@first_timee хоть я и школота, но поверь, у на...",1
1,"Да, все-таки он немного похож на него. Но мой ...",1
2,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,1
3,"RT @digger2912: ""Кто то в углу сидит и погибае...",1
4,@irina_dyshkant Вот что значит страшилка :D\nН...,1
5,ну любишь или нет? — Я не знаю кто ты бля:D ht...,1
6,"RT @SpoonLamer: Ох,900 :D ну это конечно же @t...",1
7,RT @veregijytaqo: У тебя есть ухажёр? Нет - мо...,1
8,Поприветствуем моего нового читателя @Alexey17...,1
9,Теперь у меня есть частичка Сиднея :) #Sydney ...,1


In [5]:
tweets = tweets.drop_duplicates() #важно, потому что дубликаты кажется есть

In [6]:
tweets.positive.value_counts()

1    110396
0    107044
Name: positive, dtype: int64

Убрать из данных характерные для твиттера вещи (RT, @ и т.д.)

In [7]:
tweets.tweet = tweets['tweet'].apply(lambda x: re.sub('[\Wa-zA-Z_\d]+', ' ', x) )
tweets.tweet = tweets['tweet'].apply(lambda x: x.lower().strip() )

Немного пристальнее посмотреть на записи

In [8]:
for i, v in zip(range(1,100), tweets['tweet'][50:60]):
    print(i, v)

1 а стояк теперь снова стояк вселенная вернула все на свои места
2 прифетигг солныско каг тфаи дила мну сичас фф больниську идеть к логопеду
3 да нашла фоточку в иг и нагло спиздила ничего с фотографиями меня уже статусы в вконтакте бывают
4 а у нас в нижнем слякотная погода брр но у нас офигенный привоз заходим заходим
5 короче видимо ни в какой архитектурный я не иду там капец конечно
6 ну в сделаешь тогда я дарья а ты
7 плак плак надеюсь на драфте тогда их вытяну
8 а моя кроме поорать еще просится на балкон хвост поморозить минуту посидит и назад просится
9 каждому так болеть когда к тебе друзья в гости ходят
10 мам мне так хреново выпей таблетку и на работу на следующей недели пойдем в сауну мама умеет поддержать и добавить стимула


Прелестно.
Пора переводить данные, но сначала разделю на train и test части, чтоб аугментировать только train

In [9]:
X = tweets.tweet
y = tweets.positive

In [66]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=77)

In [67]:
X_train.shape

(163080,)

### W2V augmentation part

In [12]:
from gensim.test.utils import datapath
from gensim import utils
import gensim.models

Скачать файл с векторами [отсюда](http://panchenko.me/data/dsl-backup/w2v-ru/all.norm-sz100-w10-cb0-it1-min100.w2v)

Я использую самый лёгкий файл, но в перспективе можно и более тяжёлые, всё лежит [здесь](https://nlpub.mipt.ru/Russian_Distributional_Thesaurus)

In [13]:
mipt_model = gensim.models.KeyedVectors.load_word2vec_format(
     '/home/nikitap/projects/python/augmentations_comptech/Word2Vec/mipt_vecs.w2v', binary=True, unicode_errors='ignore')

Напишу тупенькую функцию, чтоб аугментировать предложения, но сначала ещё кое-что

In [14]:
X_w2v = X_train.copy()

In [15]:
model = mipt_model

Неисследованная опция `num_trees` кажется улучшает качество поиска похожего слова; если компьютер мощный, то можно попробовать поднять значение до `100-500`

In [16]:
from gensim.similarities.index import AnnoyIndexer #вопрос, может ли работать
annoy_index = AnnoyIndexer(mipt_model, num_trees=10)

/home/nikitap/programs/anaconda3/lib/python3.7/site-packages/gensim/similarities/index.py:184: FutureWarning: The default argument for metric will be removed in future version of Annoy. Please pass metric='angular' explicitly.
  index = AnnoyIndex(num_features)


Стандартный поиск наиболее похожего слова слишком долгий, поэтому буду пользоваться [`AnnoyIndexer`](https://radimrehurek.com/gensim/auto_examples/tutorials/run_annoy.html)

In [17]:
%timeit model.most_similar('слово', topn=2, indexer=annoy_index)
%timeit model.most_similar('слово', topn=1)

220 µs ± 33.4 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
93.5 ms ± 4.38 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [19]:
print( 'Annoy:', model.most_similar('слово', topn=2, indexer=annoy_index) )
print( 'Standard:', model.most_similar('слово', topn=2) )

Annoy: [('слово', 1.0), ('произнесено', 0.6672296822071075)]
Standard: [('слово…', 0.8392893671989441), ('словечко', 0.794983983039856)]


In [28]:
import random
def sen_aug_one_change(sen):
    words = sen.split(' ')
    indexes = list( range(len(words)) )
    random.shuffle(indexes)
    new_words = list()
    for i in indexes:
        if words[i] in model:
            words[i] = model.most_similar(words[i], topn=1)[0][0]
            return ' '.join(words)
    return sen  # no replacement was possible

In [29]:
import random
def sen_aug_one_change_annoy(sen):
    words = sen.split(' ')
    indexes = list( range(len(words)) )
    random.shuffle(indexes)
    new_words = list()
    for i in indexes:
        if words[i] in model:
            words[i] = model.most_similar(words[i], topn=2, indexer=annoy_index)[1][0]
            return ' '.join(words)
    return sen  # no replacement was possible

In [30]:
import random
def sen_aug_all_changes(sen):
    words = sen.split(' ')
    for i in range(len(words)):
        if words[i] in model:
            words[i] = model.most_similar(words[i], topn=3)[1][0]
    return ' '.join(words)

In [31]:
import random
def sen_aug_all_changes_annoy(sen):
    words = sen.split(' ')
    for i in range(len(words)):
        if words[i] in model:
            words[i] = model.most_similar(words[i], topn=3, indexer=annoy_index)[1][0]
    return ' '.join(words)

Пример полностью переделанных твитов, но для быстроты будем менять по одному слову

In [35]:
i = random.randint(0, len(tweets))
print('Original tweet:', tweets.tweet[i])
print('All change words:', sen_aug_all_changes(tweets.tweet[i]) )
print('All change words (with annoy):', sen_aug_all_changes_annoy(tweets.tweet[i]) )

Original tweet: но я не обронила этих слов это что привычка или что я не поиму
All change words: не мне так промолвила таких выражений что не привычка — например не мне так поиму
All change words (with annoy): ли меня подумать усмехнулась других фраз запутали, — действительно забывчивость соответствующего действительно меня подумать поиму


Сравниваю функции по скорости работы

In [37]:
%timeit sen_aug_all_changes(X_w2v[0])
%timeit sen_aug_all_changes_annoy(X_w2v[0])
%timeit sen_aug_one_change(X_w2v[0])
%timeit sen_aug_one_change_annoy(X_w2v[0])

1.47 s ± 66.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
4.64 ms ± 635 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
98.4 ms ± 5.46 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
416 µs ± 18.7 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


С учётом количества данных в `X_w2v` = 163080, предпочитаю скорость качеству

In [39]:
%time X_add = X_w2v.apply(lambda x: sentence2sentence_shuffle(x))

CPU times: user 46.5 s, sys: 4.97 s, total: 51.5 s
Wall time: 1min 4s


Примеры того, какие аугментации получились

In [59]:
n_tweets = 15
ind = random.randint(n_tweets, len(X_w2v))
for w2v,add in zip(X_w2v.values[ind-15:ind], X_add.values[ind-n_tweets:ind]):
    print('Origin:', w2v)
    print('Aug:', add )
    print('__________')

Origin: забрал у меня айфон и смайлики айфоновские шлешь
Aug: яварски у меня айфон и смайлики айфоновские шлешь
__________
Origin: нелегкое детство будущего дизайнера
Aug: нелегкое детство перспективы дизайнера
__________
Origin: сейчас я могла бы смотреть тетрадь смерти и лопать вкусняшки но сижу под дверью и жду родителей
Aug: сейчас я могла бы смотреть тетрадь смерти и лопать вкусняшки но сижу под двери и жду родителей
__________
Origin: цагаан хоолтон гэж амьтны амь таслахыг тэвчсэн нөхдүүд байдаг уу
Aug: мангасов хоолтон гэж амьтны амь таслахыг тэвчсэн нөхдүүд байдаг уу
__________
Origin: мне урупон шею свернет лента я вас любил с
Aug: мне урупон шею помчится лента я вас любил с
__________
Origin: ахахахахахах нет это слишком пошло
Aug: ахахахахахах нет это слишком поехало…
__________
Origin: лал мне уже интересно что это за сюрприз с
Aug: лал мне уже интересно что это уговорив сюрприз с
__________
Origin: я так не хотела в школу и хотела ливень с громом и молнией что пошел ливень

In [69]:
X_train = X_w2v.append( X_add, ignore_index=True)

In [70]:
y_train = y_train.append(y_train, ignore_index=True)

In [71]:
print( X_train.shape[0], y_train.shape[0] )

326160 326160


Снова нужно вычистить дупликаты, сделаю это немного странно, т.к. нужно деражать правильные ответы

In [72]:
df_train = pd.DataFrame({'tweet': X_train.apply(lambda x: re.sub('(\W+|\d+|[A-Za-z]+|_+)', ' ', x) ), 
                         'positive': y_train})

In [73]:
df_train

,tweet,positive
0,пишу реферат без копипаст чертов антиплагиат ч...,0
1,ночнойминск и я спать всем спокойной ночи и сл...,1
2,снова попал в аварию и снова все живы блеать,0
3,как обычно ляпну не подумавши,0
4,я так устал а завтра еще и разгрузка за то под...,0
...,...,...
326155,сегодня суббота а я весь день учу этот бух уче...,0
326156,должна отметить что новый клип майли сайрус по...,0
326157,я тоже на приветливого фермера и игр типа таких,0
326158,капюшон моей куртки с синим оттенком рекламка ...,1


In [76]:
df_train = df_train.drop_duplicates()

In [77]:
len(df_train)

318322

In [78]:
X_train, y_train = df_train.tweet, df_train.positive

На всякий случай можно ещё раз почистить, ещё раз убрать все знаки препинания и цифры

Но для начала нужно слить train и test, чтоб vectorize все строки, поэтому запоминаю `split_index`

In [79]:
split_index = X_train.shape[0]
X = X_train.append(X_test).apply(lambda x: re.sub('(\W+|\d+|[A-Za-z]+|_+)', ' ', x) )

In [80]:
X[split_index-10:split_index]

326150    читай зайдешь на страницу работавшего настроен...
326151    или вот занусси да и вообще любой поляк а фран...
326152    россия московская область г егорьевск микр н с...
326153    лида разочаровала нас печальной новостью почти...
326154    думала зайду на минут поиграю в симс первой ит...
326155    сегодня суббота а я весь день учу этот бух уче...
326156    должна отметить что новый клип майли сайрус по...
326157      я тоже на приветливого фермера и игр типа таких
326158    капюшон моей куртки с синим оттенком рекламка ...
326159    жители академгородка получат скидку на оптику ...
Name: tweet, dtype: object

In [87]:
assert len(X_test) + len(X_train) == len(X)

In [88]:
len(X)

372682

Почищено, теперь векторизую

In [89]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
X_vect = vectorizer.fit_transform(X)

Снова разделяю по split индексу

In [90]:
X_vect_train, X_vect_test = X_vect[:split_index,], X_vect[split_index:,]

In [92]:
print('Размеры верны:', X_vect_train.shape[0] == y_train.shape[0] )
print('Размеры верны:', X_vect_test.shape[0] == y_test.shape[0] )

Размеры верны: True
Размеры верны: True


In [93]:
vectorizer.get_feature_names()[800:820]

['автоматы',
 'автомеханика',
 'автомобиле',
 'автомобилей',
 'автомобилем',
 'автомобили',
 'автомобилист',
 'автомобилистов',
 'автомобилисты',
 'автомобиль',
 'автомобильной',
 'автомобильном',
 'автомобильные',
 'автомобильный',
 'автомобильчик',
 'автомобиля',
 'автомойка',
 'автоновости',
 'автономии',
 'автономия']

### Go учиться

In [94]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score

In [95]:
clf = LogisticRegression(max_iter=400)

In [96]:
clf.fit(X_vect_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=400,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [97]:
y_pred = clf.predict(X_vect_test)

In [98]:
y_pred.shape

(54360,)

In [99]:
accuracy_score(y_pred, y_test)

0.7336460632818249

In [100]:
f1_score(y_pred, y_test)

0.7452001759788827

ок, хорошо

теперь можно менять параметры

In [101]:
from sklearn.model_selection import GridSearchCV

In [105]:
gs = GridSearchCV(clf, {'C': [0.75, 1, 1.25]}, cv=2, verbose=1)

In [106]:
gs.fit(X_vect_train, y_train)

Fitting 2 folds for each of 3 candidates, totalling 6 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:  2.9min finished


GridSearchCV(cv=2, error_score=nan,
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=400, multi_class='auto',
                                          n_jobs=None, penalty='l2',
                                          random_state=None, solver='lbfgs',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='deprecated', n_jobs=None, param_grid={'C': [0.75, 1, 1.25]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=1)

In [107]:
gs.best_params_

{'C': 1.25}

In [108]:
y_pred = gs.best_estimator_.predict(X_vect_test)

In [109]:
accuracy_score(y_pred, y_test)

0.7334988962472406

In [110]:
f1_score(y_pred, y_test)

0.7446009555206883